In [4]:
# !pip install cMake
# !conda install -c conda-forge dlib

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import os, cv2

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
mtcnn = MTCNN(keep_all=True, device=device)

In [138]:
new_img_dir = './input/data/train/new_imgs'
img_path = './input/data/train/images'

In [ ]:
cnt = 0

for paths in os.listdir(img_path):
    if paths[0] == '.': continue
    
    sub_dir = os.path.join(img_path, paths)
    
    for imgs in os.listdir(sub_dir):
        if imgs[0] == '.': continue
        
        img_dir = os.path.join(sub_dir, imgs)
        img = cv2.imread(img_dir)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        #mtcnn 적용
        boxes,probs = mtcnn.detect(img)
        
        # boxes 확인
        if len(probs) > 1: 
            print(boxes)
        if not isinstance(boxes, np.ndarray):
            print('Nope!')
            # 직접 crop
            img = img[100:400, 50:350, :]
        
        # boexes size 확인
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img[ymin:ymax, xmin:xmax, :]
            
        tmp = os.path.join(new_img_dir, paths)
        cnt += 1
        plt.imsave(os.path.join(tmp, imgs), img)
        
print(cnt)

In [143]:
test_info = pd.read_csv('./input/data/eval/info.csv')
path = './input/data/eval/images'
new_path = './input/data/eval/new_images'

In [ ]:
cnt = 0

if not os.path.exists(new_path):
    os.mkdir(new_path)
    
for i in test_info.values:
    img = cv2.imread(os.path.join(path, i[0]))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    #mtcnn 적용
    boxes,probs = mtcnn.detect(img)
    
    # boxes 확인
    if not isinstance(boxes, np.ndarray):
        print('Nope!')
        # 직접 crop
        img = img[100:400, 50:350, :]
    
    # boexes size 확인
    else:
        xmin = int(boxes[0, 0])-30
        ymin = int(boxes[0, 1])-30
        xmax = int(boxes[0, 2])+30
        ymax = int(boxes[0, 3])+30
        
        if xmin < 0: xmin = 0
        if ymin < 0: ymin = 0
        if xmax > 384: xmax = 384
        if ymax > 512: ymax = 512
        
        img = img[ymin:ymax, xmin:xmax, :]
        
    plt.imsave(os.path.join(new_path, i[0]), img)
    cnt += 1
    
print(cnt)